<a href="https://colab.research.google.com/github/Mat-O-Lab/MSEO/blob/main/tools/map_and_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdflib
!pip install PyGithub
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import DC, OWL, RDF, RDFS, XSD
from rdflib.plugins.sparql import prepareQuery
import github
from re import search as re_search
import ipywidgets as widgets
from IPython.display import display, clear_output

mseo = Namespace('https://purl.matolab.org/mseo/mid/')
cco = Namespace('http://www.ontologyrepository.com/CommonCoreOntologies/')
sub_classes = prepareQuery(
  "SELECT ?entity WHERE {?entity rdfs:subClassOf* ?parent}",
  initNs ={"rdf": RDF,"rdsf": RDFS}
)

def get_methods():
  #get all ttl files from methods folder of mseo repo, create dict with method name key and url to file
  g=github.Github()
  repo = g.get_repo("Mat-O-Lab/MSEO")
  methods_urls=[method.download_url for method in repo.get_contents("methods") if method.download_url.endswith('ttl')]
  methods={re_search('[^/\\&\?]+\.\w{3,4}(?=([\?&].*$|$))', url)[0].split('.')[0]: url for url in methods_urls}
  return methods

def get_ICE_classes():
  g=Graph()
  g.parse('https://raw.githubusercontent.com/CommonCoreOntology/CommonCoreOntologies/master/cco-merged/MergedAllCoreOntology-v1.3-2021-03-01.ttl',format='turtle')
  #mseo = Graph()
  g.parse("https://purl.matolab.org/mseo/mid",format='xml')
  InformtionContentEntity = URIRef("http://www.ontologyrepository.com/CommonCoreOntologies/InformationContentEntity")
  #mseo_ICE=list(mseo.query(sub_classes, initBindings={'parent': InformtionContentEntity}, initNs={ 'mseo': mseo }))
  g_ICE=list(g.query(sub_classes, initBindings={'parent': InformtionContentEntity}, initNs={ 'cco': cco , 'mseo': mseo }))
  ICE_classes=[result[0] for result in g_ICE]#+[result[0] for result in mseo_ICE]
  return ICE_classes

def get_temporal_region_classes():
  g=Graph()
  g.parse('https://raw.githubusercontent.com/CommonCoreOntology/CommonCoreOntologies/master/cco-merged/MergedAllCoreOntology-v1.3-2021-03-01.ttl',format='turtle')
  #mseo = Graph()
  g.parse("https://purl.matolab.org/mseo/mid",format='xml')
  TemporalRegionClass = URIRef("http://purl.obolibrary.org/obo/BFO_0000008")
  #mseo_ICE=list(mseo.query(sub_classes, initBindings={'parent': InformtionContentEntity}, initNs={ 'mseo': mseo }))
  g_TR=list(g.query(sub_classes, initBindings={'parent': TemporalRegionClass}, initNs={ 'cco': cco , 'mseo': mseo }))
  TR_classes=[result[0] for result in g_TR]#+[result[0] for result in mseo_ICE]
  return TR_classes

def get_methode_ICE(method_url):
  # get all the ICE individuals in the method graph
  ICE_classes=get_ICE_classes()
  TR_classes=get_temporal_region_classes()
  class_list=ICE_classes+TR_classes
  method = Graph()
  method.parse(method_url,format='turtle')
  ToMapTo=[s for s, p, o in method.triples((None,  RDF.type , None)) if o in class_list]
  return ToMapTo

def get_data_Info(data_string):
  # all Information Line individuals
  InformationLineClass = URIRef("http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine")
  ColumnClass=URIRef("http://www.w3.org/ns/csvw#Column")
  data = Graph()
  data.parse(data_string,format='json-ld')
  InfoLines=[s for s, p, o in data.triples((None,  RDF.type , None)) if o in (InformationLineClass,ColumnClass)]
  return InfoLines




     |████████████████████████████████| 379 kB 5.0 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 291 kB 5.0 MB/s 
     |████████████████████████████████| 961 kB 47.3 MB/s 


In [2]:
#@title Dialog { vertical-output: true }
# dialog
methods=get_methods()
uploader = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description='Upload'

)
p_or_sec=widgets.Box(
    [
        widgets.Label(value='primary or secondary data:'),
        widgets.RadioButtons(
            options=[
                'primary - the data describes one experiment with one specimen',
                'secondary - data table with results of multiple experiments',
            ],
            layout={'width': 'max-content'}
        )
    ]
)
file= widgets.HBox([widgets.Label(value="File:"),uploader])
method_graph = widgets.Dropdown(
    options=list(methods.keys()),
    #value=list(methods.items())[0][1],
    description='method:',
    disabled=False,
)
settings= widgets.HBox([method_graph])
button = widgets.Button(description='Start Mapping!', layout=widgets.Layout(width='200px')); 
out = widgets.Output()
def on_button_clicked(_):
  # "linking function with output"
  with out:
  # what happens when we press the button
    clear_output()
    if not uploader.value.keys():
      print('pls upload a file first')
      return
    input_file=uploader.value[list(uploader.value.keys())[0]]
    file_name = input_file['metadata']['name']
    file_data = input_file['content']
    #print(file_name,methods[method_graph.value])
    methode_ICEs=get_methode_ICE(methods[method_graph.value])
    info_lines=[(InfoLine.toPython().split('/')[-1],InfoLine) for InfoLine in get_data_Info(file_data)]
    info_lines.insert(0,('None',None))
    map_elements=list()
    for methode_ICE in methode_ICEs:
      map_elements.append(widgets.HBox([
                                        widgets.Label(value=methode_ICE.split('/')[-1]+':'),
                                        widgets.Dropdown(
                                            options=info_lines,
                                            #value='auto',
                                            #description='separator:',
                                            disabled=False,)
                                        ]))
    display(widgets.VBox(map_elements))
    print(methode_ICEs)
    
    #res = result
    #b64 = base64.b64encode(res.encode())
    #payload = b64.decode()
    # html_buttons = '''<html>
    # <head>
    # <meta name="viewport" content="width=device-width, initial-scale=1">
    # </head>
    # <body>
    # <a download="{filename}" href="data:text/json;base64,{payload}" download>
    # <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
    # </a>
    # </body>
    # </html>
    # '''
    # html_button = html_buttons.format(payload=payload,filename=metafile_name)
    # display(widgets.HTML(html_button))

button.on_click(on_button_clicked)
process = widgets.VBox([button,out])
display(file,p_or_sec,settings,process)

Box(children=(Label(value='primary or secondary data:'), RadioButtons(layout=Layout(width='max-content'), opti…

In [ ]:
input_file=uploader.value[list(uploader.value.keys())[0]]
file_name = input_file['metadata']['name']
file_data = input_file['content']
[(InfoLine.toPython().split('/')[-1],InfoLine) for InfoLine in get_data_Info(file_data)]
#print(len(raw))

[('AktuelleProbe', rdflib.term.URIRef('file:///content/AktuelleProbe')),
 ('Einspannlaenge', rdflib.term.URIRef('file:///content/Einspannlaenge')),
 ('Messlaenge', rdflib.term.URIRef('file:///content/Messlaenge')),
 ('ProbenbreiteB0', rdflib.term.URIRef('file:///content/ProbenbreiteB0')),
 ('ProbendickeA0', rdflib.term.URIRef('file:///content/ProbendickeA0')),
 ('Pruefgeschwindigkeit',
  rdflib.term.URIRef('file:///content/Pruefgeschwindigkeit')),
 ('Vorkraft', rdflib.term.URIRef('file:///content/Vorkraft')),
 ('WegNachBruch', rdflib.term.URIRef('file:///content/WegNachBruch')),
 ('EModul', rdflib.term.URIRef('file:///content/EModul')),
 ('DehnungBeiFmax', rdflib.term.URIRef('file:///content/DehnungBeiFmax')),
 ('DehnungBeiBruch', rdflib.term.URIRef('file:///content/DehnungBeiBruch')),
 ('ObereStreckgrenze',
  rdflib.term.URIRef('file:///content/ObereStreckgrenze')),
 ('Kraftmaximum', rdflib.term.URIRef('file:///content/Kraftmaximum')),
 ('Bruchkraft', rdflib.term.URIRef('file:///conte

In [ ]:
data = Graph()
data.parse(file_data,format='json-ld')
InfoLines=[(s,"is_a" ,o) for s, p, o in data.triples((None,  RDF.type , None))]# if o == 'InformationLineClass']
for item in InfoLines:
  print(item)

(rdflib.term.URIRef('file:///content/AktuelleProbe'), 'is_a', rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine'))
(rdflib.term.URIRef('file:///content/Einspannlaenge'), 'is_a', rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine'))
(rdflib.term.URIRef('file:///content/Messlaenge'), 'is_a', rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine'))
(rdflib.term.URIRef('file:///content/ProbenbreiteB0'), 'is_a', rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine'))
(rdflib.term.URIRef('file:///content/ProbendickeA0'), 'is_a', rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine'))
(rdflib.term.URIRef('file:///content/Pruefgeschwindigkeit'), 'is_a', rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/InformationLine'))
(rdflib.term.URIRef('file:///content/Vorkraft'), 'is_a', r